In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
physical_gpus = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)] 
)
logical_gpus = tf.config.list_logical_devices("GPU")

2022-07-10 20:28:50.818141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-10 20:28:50.884183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 20:28:50.884321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-07-10 20:28:50.884542: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-10 20:28:50.886066: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-10 20:28:50.887352: I tensorflow/stream_executor/pl

In [3]:
x_path = "dataset/data_sr/audio_sample.csv"
y_path = "dataset/data_sr/label_sample.csv"

In [4]:
x = pd.read_csv(x_path)
y = pd.read_csv(y_path)

In [5]:
x.head()

,檔名,特徵
0,1631855145,[[ 0.12715088 0.12715088 0.12715088 ... 0.1...
1,1619349620,[[ 0.06157185 0.06157185 0.06157185 ... 0.0...
2,1618415564,[[ 0.01064791 0.01064791 0.01064791 ... 0.0...
3,1627011377,[[ 0.15257801 0.15257801 0.15257801 ... 0.1...
4,1633141439,[[ 0.04986587 0.04986587 0.04986587 ... 0.0...


In [6]:
y.head()

,檔名,請開,請關,請保持,其他
0,1633056677,0,0,0,1
1,1648290403,0,0,0,1
2,1651108191,0,0,0,1
3,1632317974,0,0,0,1
4,1637285404,0,0,0,1


In [7]:
df = pd.merge(x,y, how="left", left_on="檔名", right_on="檔名")

In [8]:
df.head()

,檔名,特徵,請開,請關,請保持,其他
0,1631855145,[[ 0.12715088 0.12715088 0.12715088 ... 0.1...,0.0,0.0,0.0,1.0
1,1619349620,[[ 0.06157185 0.06157185 0.06157185 ... 0.0...,1.0,0.0,0.0,0.0
2,1618415564,[[ 0.01064791 0.01064791 0.01064791 ... 0.0...,0.0,0.0,0.0,1.0
3,1627011377,[[ 0.15257801 0.15257801 0.15257801 ... 0.1...,0.0,0.0,0.0,1.0
4,1633141439,[[ 0.04986587 0.04986587 0.04986587 ... 0.0...,0.0,0.0,0.0,1.0


In [9]:
x.shape

(85484, 2)

In [10]:
y.shape

(91148, 5)

In [11]:
df.shape

(91652, 6)

In [12]:
df = df.dropna()

In [13]:
X = df[df.columns[1]].values
y = df[df.columns[2:6]].values

In [14]:
X.shape, y.shape

((91110,), (91110, 4))

In [15]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [16]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'), # if x <= 0 --> 0, x > 0 --> x
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(4, activation="sigmoid")
])

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.evaluate(X_train, y_train)

2022-07-10 20:29:10.404263: W tensorflow/core/framework/op_kernel.cc:1730] OP_REQUIRES failed at cast_op.cc:123 : Unimplemented: Cast string to float is not supported
2022-07-10 20:29:10.405573: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


In [ ]:
model.evaluate(X_valid, y_valid)

In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_test, y_test)